In [16]:
import openslide


In [89]:
from openslide import (OpenSlide, OpenSlideError,
        OpenSlideUnsupportedFormatError)
import re
import sys
import PIL
import numpy as np
import os
from PIL import Image, ImageDraw
from openslide.deepzoom import DeepZoomGenerator as dz
import cv2
import math
import pandas as pd

#Import xml dependencies
from xml.dom import minidom
import xml.etree.ElementTree as ET
from math import ceil
import random

In [112]:

def reading_WSI(slide_path):
    slide = OpenSlide(slide_path)
    slide_dimensions = slide.level_dimensions
    slide_img = slide.read_region((0, 0), 1,(slide.level_dimensions[1][0], slide.level_dimensions[1][1])).convert('RGB')
    return slide_dimensions,slide_img
def annotation_conversion(slide_dimensions,coordinates):
    y_axis = slide_dimensions[1][1]/slide_dimensions[0][1]
    x_axis = slide_dimensions[1][0]/slide_dimensions[0][0]
    new_cordinate_list = []
    for j in coordinates.values():
        coordinates_list = j
    #     print(cent(j[0]*x_axis),cent(j[1]*y_axis,1))
    for k in range(len(coordinates_list)):
        new_cordinate_list.append((ceil(coordinates_list[k][0]*x_axis),ceil(coordinates_list[k][1]*y_axis)))
    return new_cordinate_list
def cropping_roi(slide_img,new_cordinate_list):
    slide1 = np.asarray(slide_img, dtype="int32")
    mask = np.zeros((slide1.shape[0], slide1.shape[1]))
    cv2.fillConvexPoly(mask, np.array(new_cordinate_list), 1)
    mask = mask.astype(np.bool)
    out = np.zeros_like(slide1)
    out[mask] = slide1[mask]
    cv2.imwrite("example.png",np.array(out))
    return np.array(out)
def parse(inputfile,regions, vertices):

    
    tree = ET.parse(inputfile)
    root = tree.getroot()

    #dictionary of point coordinates
    images = {}
    for region in root.iter(regions):
        coordinates = []
        for vertex in region.iter(vertices):
            temp = [int(i) for i in vertex.attrib.values()]
            coordinates.append(temp)
        
        #append the coordinates to images dictionary
        images.update({region.attrib["Id"]: np.array(coordinates)})
    
    return images
def cropping_main():
    slide_dimensions,slide_img = reading_WSI(r'C:\Users\avinash\Dropbox\Data\Moderately differentiated\1003954.svs')
    coordinates = parse(r"C:\Users\avinash\Dropbox\Data\Moderately differentiated\1003954.xml",regions = 'Region', vertices = 'Vertex')
    new_cordinate_list = annotation_conversion(slide_dimensions,coordinates)
    new_image = cropping_roi(slide_img,new_cordinate_list)
    return new_image 

def stainremover_small_patch_remover(img):
    cleaned_Images = []

    # img = cv2.imread(inputimage)
    # img = cv2.imread(inputimage,0)
    if len(img) < 256 or len(img[0]) < 256:
        #         print("lessdemsion")
        return None
        # print(inputimage)
        # os.remove(inputimage)
    else:
        # print('here')
        Xb = []
        Xg = []
        Xr = []
        for i in range(len(img)):
            Xb.append(np.mean(img[i][:, 0]))
            Xg.append(np.mean(img[i][:, 1]))
            Xr.append(np.mean(img[i][:, 2]))
        #     print(inputimage)
        #     print(np.mean(Xr),np.mean(Xg),np.mean(Xb))

        if np.mean(Xr) < 70 or np.mean(Xr) > 220:
            # print(np.mean(Xr))
            # print("red")
            #             print(inputimage.split('/')[-1])
            return None
            # print(inputimage)
            # os.remove(inputimage)

        elif np.mean(Xg) < 80 or np.mean(Xg) > 200:
            # print("green")
            return None
            # print(inputimage)
            # os.remove(inputimage)

        elif np.mean(Xb) < 100 or np.mean(Xb) > 210:
            # print("blue")
            return None
        else:
            # print('here')

            cv2.imwrite("temp0.png", img)
            img_bg = cv2.imread("temp0.png", 0)
            # img_bg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            if (img_bg.mean() < 215) and (img_bg.mean() > 50):

                return img

            else:
                # print((img_bg.mean()))

                # print("Blackandwhite")
                return None
def random_croping(img, tile_size):
    img_shape = img.shape
    # tile_size = (256, 256)
    x_point = random.randint(0, img_shape[0])
    y_point = random.randint(0, img_shape[1])
    try:
        # if (cropped_img.mean() > 50) and (cropped_img.mean() < 180) and (len(list(set(cropped_img.flatten()))) > 50):
        cropped_image = img[y_point:y_point + 256, x_point:x_point + 256]

        cleaned_image = stainremover_small_patch_remover(cropped_image)

    except:
        None
#     cv2.imwrite('1.png',cleaned_image)
    return cleaned_image
def path_extraction(img, path):
    tile_size = (256, 256)

    len_list = []
    for i in range(100000):

        patchs = random_croping(img, tile_size)
        
        if len(len_list) < 2000:
            print(patchs)
            
            if patchs is None:
                None
            else:
                cv2.imwrite('%s/%s.png' % (path, i), patchs)
                len_list.append(i)

        else:
            
            return
    return
# os.mkdir('example')

path = "example/" 

slide = cropping_main()

path_extraction(slide, path)


[[[210 205 195]
  [229 221 215]
  [153 118 106]
  ...
  [186 190 205]
  [216 217 217]
  [156 156 171]]

 [[191 181 170]
  [251 239 220]
  [191 148 129]
  ...
  [181 186 203]
  [223 231 225]
  [194 192 195]]

 [[172 170 168]
  [236 229 221]
  [188 143 135]
  ...
  [208 194 198]
  [224 221 216]
  [226 211 214]]

 ...

 [[241 244 226]
  [230 235 234]
  [228 228 228]
  ...
  [238 224 214]
  [197 197 198]
  [175 180 167]]

 [[231 221 222]
  [237 232 243]
  [235 237 236]
  ...
  [185 181 204]
  [166 174 187]
  [168 152 162]]

 [[236 230 204]
  [242 243 221]
  [232 238 226]
  ...
  [169 179 197]
  [155 172 184]
  [162 151 165]]]
None
None
None
[[[ 64  39  36]
  [ 33  12  12]
  [ 62  36  13]
  ...
  [234 228 220]
  [232 226 216]
  [221 214 210]]

 [[164 135 114]
  [109  74  78]
  [107  60  56]
  ...
  [230 225 223]
  [219 213 210]
  [210 203 199]]

 [[214 183 146]
  [215 191 156]
  [215 186 164]
  ...
  [208 203 199]
  [211 206 204]
  [228 222 213]]

 ...

 [[236 236 230]
  [228 225 230]
  [23

None
[[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [242 241 241]
  [242 241 241]
  [242 241 241]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [242 241 241]
  [242 241 241]
  [242 241 241]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [242 241 241]
  [242 241 241]
  [242 241 241]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [241 241 241]
  [241 241 241]
  [241 241 241]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [241 241 241]
  [241 241 241]
  [241 241 241]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [241 241 239]
  [241 241 239]
  [241 241 239]]]
None
None
None
None
None
None
None
None
None
None
None
None
None
[[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0

None
None
None
None
None
None
None
None
None
[[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 ...

 [[242 243 242]
  [242 243 242]
  [242 243 242]
  ...
  [243 243 243]
  [243 243 243]
  [243 243 243]]

 [[242 243 242]
  [242 243 242]
  [242 243 242]
  ...
  [243 243 243]
  [243 243 243]
  [243 243 243]]

 [[242 243 242]
  [242 243 242]
  [242 243 242]
  ...
  [243 243 243]
  [243 243 243]
  [243 243 243]]]
None
None
[[[241 239 239]
  [241 239 239]
  [241 239 239]
  ...
  [228 218 214]
  [240 236 232]
  [226 221 220]]

 [[241 239 239]
  [241 239 239]
  [241 239 239]
  ...
  [234 219 208]
  [234 230 226]
  [236 233 219]]

 [[241 239 239]
  [241 239 239]
  [241 239 239]
  ...
  [220 211 210]
  [212 203 205]
  [231 226 221]]

 ..

  [241 241 240]]]
None
[[[219 199 193]
  [182 156 104]
  [ 84  47  42]
  ...
  [239 239 239]
  [239 239 239]
  [239 239 239]]

 [[228 214 190]
  [164 124  85]
  [ 72  42  29]
  ...
  [239 239 238]
  [239 239 238]
  [239 239 238]]

 [[232 231 195]
  [124  85  75]
  [ 53  40  25]
  ...
  [239 239 238]
  [239 239 238]
  [239 239 238]]

 ...

 [[ 76  35  19]
  [ 60  35   7]
  [ 99  44  20]
  ...
  [ 45  30  20]
  [ 76  27  16]
  [124  65  36]]

 [[ 31  18  30]
  [ 40  29  18]
  [ 97  43  42]
  ...
  [ 66  27  17]
  [102  49  17]
  [112  60  27]]

 [[106  46  49]
  [132  73  38]
  [125  82  56]
  ...
  [ 84  31  22]
  [ 78  39  19]
  [ 64  35  20]]]
None
None
None
None
[[[189 191 192]
  [222 225 225]
  [244 247 244]
  ...
  [240 239 239]
  [239 238 239]
  [239 238 239]]

 [[151 153 167]
  [200 202 210]
  [244 245 245]
  ...
  [240 239 239]
  [239 238 239]
  [239 238 239]]

 [[131 137 162]
  [161 163 181]
  [203 202 210]
  ...
  [240 239 239]
  [239 238 239]
  [239 238 239]]

 ...

 [[ 91 11

[[[245 236 238]
  [240 237 238]
  [239 240 238]
  ...
  [241 239 239]
  [241 239 239]
  [241 239 239]]

 [[237 238 240]
  [240 235 237]
  [243 239 237]
  ...
  [241 239 239]
  [241 239 239]
  [241 239 239]]

 [[230 238 239]
  [239 236 238]
  [247 239 238]
  ...
  [241 239 239]
  [241 239 239]
  [241 239 239]]

 ...

 [[207 200 188]
  [198 176 163]
  [181 169 155]
  ...
  [241 239 239]
  [241 239 239]
  [241 239 239]]

 [[192 182 166]
  [191 162 152]
  [182 161 159]
  ...
  [241 239 239]
  [241 239 239]
  [241 239 239]]

 [[188 181 167]
  [188 171 155]
  [189 182 164]
  ...
  [241 239 239]
  [241 239 239]
  [241 239 239]]]
[[[184 187 169]
  [129 108  99]
  [136  91  58]
  ...
  [ 99  45  21]
  [ 82  37  22]
  [ 63  27  21]]

 [[184 163 171]
  [101  87  79]
  [ 44  24  12]
  ...
  [ 63  26  15]
  [ 57  28  14]
  [ 50  26  11]]

 [[178 129  94]
  [146 103  69]
  [ 78  44  26]
  ...
  [ 61  37  12]
  [ 51  31   8]
  [ 34  11  17]]

 ...

 [[212 187 162]
  [234 225 214]
  [244 246 244]
  ..

...

[[[240 241 240]
  [240 241 239]
  [240 240 238]
  ...
  [ 71  39  20]
  [ 49  26  10]
  [ 85  41  32]]

 [[240 239 239]
  [240 239 239]
  [240 239 238]
  ...
  [ 89  47  26]
  [111  48  17]
  [134  60  29]]

 [[240 238 239]
  [240 238 239]
  [240 238 239]
  ...
  [ 84  40  25]
  [ 72  33  21]
  [ 60  42  20]]

 ...

 [[241 241 241]
  [241 241 241]
  [241 241 241]
  ...
  [239 238 237]
  [239 238 236]
  [239 237 236]]

 [[241 241 241]
  [241 241 241]
  [241 241 241]
  ...
  [239 238 237]
  [239 238 237]
  [239 238 237]]

 [[241 241 241]
  [241 241 241]
  [241 241 241]
  ...
  [240 239 238]
  [240 238 238]
  [240 238 238]]]
None
None
None
None
None
None
None
[[[229 230 228]
  [233 229 227]
  [239 233 231]
  ...
  [161 126  93]
  [119  78  50]
  [125  78  45]]

 [[229 225 228]
  [227 224 225]
  [230 228 228]
  ...
  [125  86  48]
  [142  91  46]
  [ 96  60  42]]

 [[235 231 236]
  [234 230 233]
  [233 230 232]
  ...
  [112  64  34]
  [ 84  43  41]
  [124  74  46]]

 ...

 [[247 247 248]
 

[[[242 242 241]
  [242 242 241]
  [242 242 241]
  ...
  [216 204 194]
  [242 232 221]
  [252 224 224]]

 [[242 242 241]
  [242 242 241]
  [242 242 241]
  ...
  [230 224 219]
  [240 246 210]
  [218 218 175]]

 [[242 242 241]
  [242 242 241]
  [242 242 241]
  ...
  [232 195 151]
  [181 141 100]
  [120  85  62]]

 ...

 [[239 237 232]
  [238 234 234]
  [236 229 235]
  ...
  [ 79  32  12]
  [118  74  58]
  [196 162 131]]

 [[238 237 228]
  [236 235 231]
  [234 232 232]
  ...
  [ 54  47  36]
  [154 120 105]
  [221 183 174]]

 [[237 235 227]
  [236 234 229]
  [233 232 230]
  ...
  [124  67  52]
  [209 169 135]
  [209 207 199]]]
None
[[[134 124 123]
  [103 127 151]
  [146 150 175]
  ...
  [107  99  81]
  [ 74  33  35]
  [142  94  52]]

 [[113  95  93]
  [126 131 156]
  [164 165 192]
  ...
  [124 100  75]
  [ 68  38  49]
  [ 85  65  43]]

 [[160 137 125]
  [197 178 190]
  [220 206 221]
  ...
  [ 84  70  46]
  [ 51  29  26]
  [ 27  17   7]]

 ...

 [[144 102  64]
  [112  75  47]
  [118  82  53]

  [243 243 243]]]
None
[[[226 231 210]
  [236 237 229]
  [240 232 227]
  ...
  [222 206 213]
  [188 175 183]
  [167 146 143]]

 [[209 163 140]
  [239 220 221]
  [231 227 229]
  ...
  [196 184 195]
  [202 195 192]
  [184 173 172]]

 [[107  89  66]
  [229 226 216]
  [239 228 200]
  ...
  [188 154 120]
  [164 133 105]
  [125 104  81]]

 ...

 [[ 69  33  25]
  [107  52  35]
  [113  66  34]
  ...
  [ 84  60  40]
  [ 77  58  49]
  [118  99  87]]

 [[ 51  24  22]
  [106  60  33]
  [139  72  39]
  ...
  [134 108  71]
  [109  85  63]
  [112  85  67]]

 [[ 47  25  22]
  [ 60  37  20]
  [108  51  27]
  ...
  [155 120 115]
  [132 104  90]
  [120  96  77]]]
None
None
[[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 ...

 [[243 243 243]
  [

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
[[[131 134 153]
  [219 214 209]
  [235 234 233]
  ...
  [239 230 244]
  [231 220 218]
  [241 243 228]]

 [[195 194 200]
  [231 227 235]
  [236 236 234]
  ...
  [237 242 221]
  [240 235 233]
  [232 231 232]]

 [[244 236 229]
  [249 238 243]
  [241 238 236]
  ...
  [234 233 222]
  [233 230 230]
  [230 229 230]]

 ...

 [[240 237 235]
  [238 238 236]
  [220 212 212]
  ...
  [240 238 236]
  [232 226 219]
  [242 233 228]]

 [[235 232 233]
  [236 237 235]
  [223 217 217]
  ...
  [229 230 227]
  [234 233 218]
  [238 233 230]]

 [[234 230 233]
  [235 231 233]
  [227 231 227]
  ...
  [227 224 225]
  [228 230 215]
  [209 210 210]]]
None
None
None
None
None
None
[[[242 242 241]
  [242 242 241]
  [242 242 241]
  ...
  [242 243 242]
  [242 243 242]
  [242 243 242]]

 [[242 242 241]
  [242 242 241]
  [242 242 241]
  ...
  [242 243 242]
  [242 243 242]
  [242 243 242]]

 [[242 242 241]
  [242 242 

  [242 243 242]]]
None
None
None
None
[[[134  74  50]
  [121  91  61]
  [138  95  93]
  ...
  [ 49  24  24]
  [ 68  18  18]
  [ 87  49  37]]

 [[124  78  53]
  [121 106  60]
  [120  82  69]
  ...
  [110  69  60]
  [ 88  68  63]
  [130 114 103]]

 [[107  68  54]
  [120  86  57]
  [124  75  56]
  ...
  [184 138 139]
  [173 144 147]
  [190 166 170]]

 ...

 [[230 230 220]
  [235 229 209]
  [211 219 202]
  ...
  [ 85  57  27]
  [100  72  44]
  [137  98  66]]

 [[230 228 201]
  [243 233 229]
  [223 192 209]
  ...
  [ 95  45  18]
  [136  67  50]
  [161 118  91]]

 [[180 146 100]
  [230 226 195]
  [241 230 229]
  ...
  [112  65  20]
  [146  96  55]
  [153 139  89]]]
None
None
None
None
None
None
[[[230 213 193]
  [242 223 194]
  [196 181 166]
  ...
  [251 245 233]
  [243 230 219]
  [229 224 201]]

 [[235 240 219]
  [243 202 195]
  [217 176 183]
  ...
  [244 233 202]
  [210 196 163]
  [211 177 129]]

 [[241 234 225]
  [215 195 188]
  [209 203 206]
  ...
  [212 195 173]
  [146 129 109]
  [ 59  

In [46]:
x = slide.level_dimensions

In [51]:
mask =cv2.boundingRect(np.array(new_cordinate_list))

In [52]:
x,y,w,h = mask
# croped = img[y:y+h, x:x+w].copy()
width = slide.level_dimensions[1][0]
height = slide.level_dimensions[1][1]


In [77]:

# slide1.save("example.png")

# slide1 = slide1[y:y+h, x:x+w]
# cv2.imwrite("example.png",slide1)

In [78]:
slide1 = slide1
# cv2.imwrite("example.png",np.array(slide1))


True

In [63]:
slide.level_dimensions

((175168, 28725), (43792, 7181), (10948, 1795), (2737, 448))